In [85]:
import os

import pandas as pd
import numpy as np
import pandas_gbq

from google.cloud import bigquery
from google.oauth2 import service_account

import sqlite3

from tqdm import tqdm
 
 
for i in tqdm(range(int(9e6))):
    pass

100%|██████████| 9000000/9000000 [00:01<00:00, 7801261.43it/s]


In [ ]:
pip install tqdm

In [56]:
service_path = ""
service_file = 'umt-msba-gg-key.json'

gbq_proj_id = 'umt-msba'

private_key = service_path + service_file   

credentials = service_account.Credentials.from_service_account_file(private_key)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)
for item in client.list_datasets() :
    print(item.full_dataset_id)

umt-msba:dram_shop
umt-msba:transactions
umt-msba:wedge_example
umt-msba:wedge_transactions


In [61]:
query="""

SELECT
 DATE(datetime) AS formatted_date,
 EXTRACT(hour FROM TIMESTAMP(datetime)) AS hour,
 ROUND(SUM(total), 2) AS spend,
 COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no,trans_no))) AS trans,
 SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department not in (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY formatted_date, hour 
ORDER BY formatted_date, hour;

"""

In [62]:
conn = sqlite3.connect("reporting.db")

In [63]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

c:\ADA\Wedge\wedge-project\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [67]:
holder.head()

,formatted_date,hour,spend,trans,items
0,2010-01-01,9,1006.28,36,245
1,2010-01-01,10,3128.55,82,913
2,2010-01-01,11,4001.66,118,1108
3,2010-01-01,12,3886.51,124,1143
4,2010-01-01,13,4654.52,154,1365


In [68]:
holder.to_sql("date_hour",conn,if_exists="replace",index=False)

39330

In [69]:
conn.close()

In [71]:
query="""
SELECT
    card_no,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
    ROUND(SUM(total)) AS sales,
    COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no,trans_no))) AS trans,
    SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*`
WHERE department not in (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY
    card_no, year, month
ORDER BY
    card_no, year, month;
"""

In [72]:
conn = sqlite3.connect("reporting.db")

In [73]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

c:\ADA\Wedge\wedge-project\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [74]:
holder.head()

,card_no,year,month,sales,trans,items
0,3.0,2010,1,505049.0,21675,145549
1,3.0,2010,2,490014.0,21113,141309
2,3.0,2010,3,563682.0,25755,164673
3,3.0,2010,4,563672.0,26264,162704
4,3.0,2010,5,578451.0,27371,165280


In [75]:
holder.to_sql("owner_year_month",conn,if_exists="replace",index=False)

808896

In [76]:
conn.close()

In [82]:
query="""
SELECT
    LPAD(CAST(upc AS STRING), 14) AS upc,
    description,
    trans.department AS department_number,
    dept_name AS department_name,
    EXTRACT(YEAR FROM datetime) AS year,
    EXTRACT(MONTH FROM datetime) AS month,
    ROUND(SUM(total),2) AS sales,
    COUNT(DISTINCT(CONCAT(EXTRACT(date FROM datetime),register_no,emp_no,trans_no))) AS transactions,
    SUM(CASE WHEN trans_status in ("R", "V") THEN -1 ELSE 1 END) AS items
FROM `umt-msba.transactions.transArchive_*` AS trans
JOIN `umt-msba.transactions.department_lookup` AS d
ON d.department = trans.department
WHERE trans.department not IN (0, 15) AND
(trans_status is null OR
trans_status in ("R","V", " "))
GROUP BY
    upc, description, trans.department, dept_name, year, month
ORDER BY
    description, year, month;
"""


In [83]:
conn = sqlite3.connect("reporting.db")

In [86]:
holder = pandas_gbq.read_gbq(query,project_id=gbq_proj_id,credentials=credentials)

c:\ADA\Wedge\wedge-project\.venv\Lib\site-packages\google\cloud\bigquery\table.py:2153: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [87]:
holder.head()

,upc,description,department_number,department_name,year,month,sales,transactions,items
0,0059484110076,* Manufacturers Coupon,11.0,PERSONAL CARE,2010,1,1.00,1,-1
1,0057468211475,* Manufacturers Coupon,1.0,PACKAGED GROCERY,2010,1,0.75,1,-1
2,0058131210014,* Manufacturers Coupon,5.0,CHEESE,2010,1,4.99,1,-1
3,0053291700176,* Manufacturers Coupon,1.0,PACKAGED GROCERY,2010,1,2.00,1,-2
4,0051868700009,* Manufacturers Coupon,6.0,FROZEN,2010,1,2.00,1,-1


In [88]:
holder.to_sql("prod_desc_year_month",conn,if_exists="replace",index=False)

1184023

In [89]:
conn.close()